<a href="https://colab.research.google.com/github/Zardian18/Jokes_Generator-LSTM/blob/master/LSTM_jokes_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/Zardian18/helper-functions-colab/master/helper.py

--2024-01-11 23:03:24--  https://raw.githubusercontent.com/Zardian18/helper-functions-colab/master/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17171 (17K) [text/plain]
Saving to: ‘helper.py’

helper.py           100%[===================>]  16.77K  --.-KB/s    in 0s      

2024-01-11 23:03:24 (37.0 MB/s) - ‘helper.py’ saved [17171/17171]



In [2]:
!pip install kaggle

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
! mkdir ~/.kaggle

In [5]:
!cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/kaggle.json

In [6]:
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
! kaggle datasets download -d thedevastator/one-million-reddit-jokes

 79% 73.0M/93.0M [00:01<00:00, 62.6MB/s]
100% 93.0M/93.0M [00:01<00:00, 81.4MB/s]


In [8]:
! unzip one-million-reddit-jokes.zip

Archive:  one-million-reddit-jokes.zip
  inflating: one-million-reddit-jokes.csv  


In [9]:
import numpy as np
import json
import re
import string
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, losses
import pandas as pd

In [10]:
jokes_df = pd.read_csv("one-million-reddit-jokes.csv")
jokes_df.head(5)

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score
0,post,ftbp1i,2qh72,jokes,False,1585785543,https://old.reddit.com/r/Jokes/comments/ftbp1i...,self.jokes,NaN,My corona is covered with foreskin so it is no...,I am soooo glad I'm not circumcised!,2
1,post,ftboup,2qh72,jokes,False,1585785522,https://old.reddit.com/r/Jokes/comments/ftboup...,self.jokes,NaN,It's called Google Sheets.,Did you know Google now has a platform for rec...,9
2,post,ftbopj,2qh72,jokes,False,1585785508,https://old.reddit.com/r/Jokes/comments/ftbopj...,self.jokes,NaN,The vacuum doesn't snore after sex.\r\n\r\n&am...,What is the difference between my wife and my ...,15
3,post,ftbnxh,2qh72,jokes,False,1585785428,https://old.reddit.com/r/Jokes/comments/ftbnxh...,self.jokes,NaN,[removed],My last joke for now.,9
4,post,ftbjpg,2qh72,jokes,False,1585785009,https://old.reddit.com/r/Jokes/comments/ftbjpg...,self.jokes,NaN,[removed],The Nintendo 64 turns 18 this week...,134


In [11]:
jokes_list=[]

jokes_list.append(jokes_df["title"])
jokes_list[:5]

[0                      I am soooo glad I'm not circumcised!
 1         Did you know Google now has a platform for rec...
 2         What is the difference between my wife and my ...
 3                                     My last joke for now.
 4                     The Nintendo 64 turns 18 this week...
                                 ...                        
 999993                         With Zyan Malik leaving 1D..
 999994                Why did the exterminator go to Italy?
 999995    What did Arnold Schwarzenegger say when invite...
 999996                                        The Moth Joke
 999997                    Why did the pig go to the fridge?
 Name: title, Length: 999998, dtype: object]

In [12]:
len(jokes_list[0])

999998

In [13]:
jokes_list_final=[]

for i in range(len(jokes_list[0])):
  jokes_list_final.append(jokes_list[0][i])

jokes_list_final[:5]

["I am soooo glad I'm not circumcised!",
 'Did you know Google now has a platform for recording your bowel movements?',
 'What is the difference between my wife and my vacuum?',
 'My last joke for now.',
 'The Nintendo 64 turns 18 this week...']

In [14]:
jokes_list_final[25]

"Tee Pee a House on, April Fool's, 2020"

In [15]:
def pad_punctuation(s):
  s = re.sub(f"([{string.punctuation}])", r' \1', s)
  s = re.sub(" +", " ", s)
  return s

In [16]:
text_data = [pad_punctuation(x) for x in jokes_list_final]

In [17]:
text_data[25]

"Tee Pee a House on , April Fool 's , 2020"

In [18]:
text_ds = tf.data.Dataset.from_tensor_slices(text_data).batch(128).shuffle(1000)

In [19]:
text_ds

<_ShuffleDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>

In [20]:
sum =0
for i in range(len(text_data)):
  words = text_data[i].split()
  sum += len(words)

print(f"Average words per joke: {sum/len(text_data)}")

Average words per joke: 10.840525681051362


In [43]:
vectorize_layer = layers.TextVectorization(
    standardize= "lower",
    max_tokens = 15000,
    output_mode = "int",
    output_sequence_length = 16+1,
)

In [44]:
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

In [45]:
vocab[:10]

['', '[UNK]', '.', 'a', 'the', '?', 'i', 'to', 'what', 'you']

In [46]:
vocab[-10:]

['litre',
 'literacy',
 'listing',
 'leaky',
 'laundering',
 'lashes',
 'lace',
 'koch',
 'kane',
 'juvenile']

In [47]:
vectorize_layer((text_data[0]))

<tf.Tensor: shape=(17,), dtype=int64, numpy=
array([    6,   237, 11652,  2850,     6,    83,    93,  3151,    47,
           0,     0,     0,     0,     0,     0,     0,     0])>

In [49]:
vocab[6], vocab[237], vocab[11652], vocab[2850], vocab[83], vocab[93]

('i', 'am', 'soooo', 'glad', "'m", 'not')

## Creating Training Set

In [50]:
def prepare_inputs(text):
  text = tf.expand_dims(text, axis=-1)
  tokenized_sentences = vectorize_layer(text)
  x = tokenized_sentences[:,:-1]
  y = tokenized_sentences[:,1:]

  return x,y

In [51]:
train_ds = text_ds.map(prepare_inputs)

In [52]:
train_ds

<_MapDataset element_spec=(TensorSpec(shape=(None, 16), dtype=tf.int64, name=None), TensorSpec(shape=(None, 16), dtype=tf.int64, name=None))>

In [71]:
inputs = layers.Input(shape=(None, ), dtype="int32")
x = layers.Embedding(15000, 100)(inputs)
x = layers.LSTM(128, return_sequences=True)(x)
x = layers.LSTM(128, return_sequences=True)(x)

outputs = layers.Dense(15000, activation="softmax")(x)

lstm = models.Model(inputs, outputs)
lstm.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 100)         1500000   
                                                                 
 lstm_4 (LSTM)               (None, None, 128)         117248    
                                                                 
 lstm_5 (LSTM)               (None, None, 128)         131584    
                                                                 
 dense_2 (Dense)             (None, None, 15000)       1935000   
                                                                 
Total params: 3683832 (14.05 MB)
Trainable params: 3683832 (14.05 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [72]:
lstm.compile(loss= losses.SparseCategoricalCrossentropy(),
             optimizer="adam")

In [110]:
class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }  # <1>

    def sample_from(self, probs, temperature):  # <2>
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]  # <3>
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:  # <4>
            x = np.array([start_tokens])
            y = self.model.predict(x, verbose=0)  # <5>
            sample_token, probs = self.sample_from(y[0][-1], temperature)  # <6>
            info.append({"prompt": start_prompt, "word_probs": probs})
            start_tokens.append(sample_token)  # <7>
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\ngenerated text:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        wordx = np.random.randint(1, 10000)
        vocabx= vocab[wordx]
        self.generate(f"{vocabx}", max_tokens=100, temperature=1.0)

In [111]:
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath="./checkpoint/checkpoint.ckpt",
    save_weights_only=True,
    save_freq="epoch",
    verbose=0,
)

tensorboard_callback = callbacks.TensorBoard(log_dir="./logs")

text_generator = TextGenerator(vocab)

In [113]:
lstm.fit(
    train_ds,
    epochs=10,
    callbacks=[model_checkpoint_callback, tensorboard_callback, text_generator],
)

Epoch 1/10
   5/7813 [..............................] - ETA: 5:16 - loss: 2.3568

7811/7813 [============================>.] - ETA: 0s - loss: 2.2591
generated text:
questioning  that r /jokes writer is like popeye 's daughter . 

7813/7813 [==============================] - 175s 22ms/step - loss: 2.2590
Epoch 2/10
7812/7813 [============================>.] - ETA: 0s - loss: 2.1839
generated text:
commonly  people cause a common loan jokes ! joke . . . 

7813/7813 [==============================] - 174s 22ms/step - loss: 2.1839
Epoch 3/10
7811/7813 [============================>.] - ETA: 0s - loss: 2.1334
generated text:
scariest  man at a party 

7813/7813 [==============================] - 173s 22ms/step - loss: 2.1333
Epoch 4/10
7812/7813 [============================>.] - ETA: 0s - loss: 2.0970
generated text:
slice  cup from political humor goes into heaven . 

7813/7813 [==============================] - 173s 22ms/step - loss: 2.0970
Epoch 5/10
7813/7813 [==============================] - ETA: 0s - loss: 2.0692
generated text:
butcher  's dreams 

7813/7813 [=

In [114]:
def print_probs(info, vocab, top_k=5):
    for i in info:
        print(f"\nPROMPT: {i['prompt']}")
        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")

In [155]:
info = text_generator.generate(
    "black", max_tokens=10, temperature=1
)


generated text:
black for disappointment 



In [158]:
info = text_generator.generate(
    "black", max_tokens=10, temperature=1
)


generated text:
black history jokes are like bed 



In [159]:
print_probs(info, vocab)


PROMPT: black
people:   	19.06%
lives:   	7.51%
guy:   	6.23%
jokes:   	4.72%
humor:   	3.44%
--------


PROMPT: black history
:   	10.57%
teacher:   	9.71%
joke:   	7.72%
is:   	4.31%
[UNK]:   	4.21%
--------


PROMPT: black history jokes
:   	43.21%
.:   	11.73%
are:   	6.93%
::   	5.84%
-:   	4.64%
--------


PROMPT: black history jokes are
like:   	20.86%
not:   	6.7%
the:   	6.69%
so:   	3.96%
great:   	3.8%
--------


PROMPT: black history jokes are like
food:   	6.48%
[UNK]:   	5.09%
women:   	3.82%
a:   	3.4%
the:   	2.75%
--------


PROMPT: black history jokes are like bed
:   	18.55%
.:   	14.73%
and:   	10.1%
,:   	4.58%
[UNK]:   	3.15%
--------



In [160]:
import numpy as np

def generate_text_from_model(model, index_to_word, start_prompt, max_tokens=100, temperature=1.0):
    word_to_index = {word: index for index, word in enumerate(index_to_word)}

    def sample_from(probs, temperature):
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    start_tokens = [word_to_index.get(x, 1) for x in start_prompt.split()]
    sample_token = None
    generated_text = ""

    while len(start_tokens) < max_tokens and sample_token != 0:
        x = np.array([start_tokens])
        y = model.predict(x, verbose=0)
        sample_token, _ = sample_from(y[0][-1], temperature)
        start_tokens.append(sample_token)
        generated_text += " " + index_to_word[sample_token]

    return generated_text.strip()

In [161]:
model = lstm
index_to_word = vocab

In [84]:
start_prompt = "i like  "

generated_text = generate_text_from_model(model, index_to_word, start_prompt, temperature=0.3)
print(f"Generated text:\n{start_prompt + generated_text}")

Generated text:
i like  my women like i like my coffee


In [166]:
wordx = np.random.randint(1, 10000)
vocabx= vocab[wordx]
start_prompt = vocabx +" "

generated_text = generate_text_from_model(model, index_to_word, start_prompt, temperature=1)
print(f"Generated text:\n{start_prompt + generated_text}")

Generated text:
tin did u hear a joke about food poisoning ?


In [102]:
wordx = np.random.randint(1, 10000)
vocabx= vocab[wordx]
start_prompt = vocabx +" "

generated_text = generate_text_from_model(model, index_to_word, start_prompt, temperature=0.8)
print(f"Generated text:\n{start_prompt + generated_text}")

Generated text:
nailed a good day , but i 'm supposed to love it in the band


In [104]:
wordx = np.random.randint(1, 10000)
vocabx= vocab[wordx]
start_prompt = vocabx +" "

generated_text = generate_text_from_model(model, index_to_word, start_prompt, temperature=0.8)
print(f"Generated text:\n{start_prompt + generated_text}")

Generated text:
noticed me my wife made me . . .


In [183]:
wordx = np.random.randint(1, 10000)
vocabx= vocab[wordx]
start_prompt = vocabx +" "

generated_text = generate_text_from_model(model, index_to_word, start_prompt, temperature=0.8)
print(f"Generated text:\n{start_prompt + generated_text}")

Generated text:
historians call me a minecraft movie .


In [197]:
wordx = np.random.randint(1, 15000)
vocabx= vocab[wordx]
start_prompt = vocabx +" "

generated_text = generate_text_from_model(model, index_to_word, start_prompt, temperature=0.8)
print(f"Generated text:\n{start_prompt + generated_text}")

Exception ignored in: <function AtomicFunction.__del__ at 0x7cfc5363f5b0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/polymorphic_function/atomic_function.py", line 286, in __del__
    def __del__(self):
KeyboardInterrupt: 


Generated text:
pistol interviewer : sir , can i get some money for a [UNK] that ?


In [212]:
wordx = np.random.randint(1, 15000)
vocabx= vocab[wordx]
start_prompt = vocabx +" "

generated_text = generate_text_from_model(model, index_to_word, start_prompt, temperature=0.8)
print(f"Generated text:\n{start_prompt + generated_text}")

Generated text:
forks are made in the car if they were taking a bath .


In [221]:
info = text_generator.generate(
    "my brother", max_tokens=10, temperature=1
)


generated text:
my brother 's law degree walks into a bar 



In [230]:
info = text_generator.generate(
    "my brother", max_tokens=10, temperature=0.2
)


generated text:
my brother said he wanted to be a comedian .



In [275]:
info = text_generator.generate(
    "why", max_tokens=20, temperature=0.8
)


generated text:
why did the duck get arrested for a crime scene ? 



In [276]:
info = text_generator.generate(
    "because", max_tokens=10, temperature=0.8
)


generated text:
because i named trump 'the kkk movie 



In [286]:
info = text_generator.generate(
    "because", max_tokens=10, temperature=0.8
)


generated text:
because trump 's what makes it do ? 



In [271]:
info = text_generator.generate(
    "why", max_tokens=10, temperature=0.8
)


generated text:
why are the 'dark and black people so young ?



In [272]:
info = text_generator.generate(
    "because", max_tokens=20, temperature=1
)


generated text:
because they just can be 



In [287]:
print_probs(info, vocab)


PROMPT: because
i:   	50.73%
of:   	10.5%
you:   	10.2%
it:   	8.3%
they:   	2.7%
--------


PROMPT: because trump
is:   	32.95%
's:   	20.93%
and:   	6.48%
,:   	4.93%
has:   	4.73%
--------


PROMPT: because trump 's
[UNK]:   	13.41%
the:   	6.36%
immigration:   	3.75%
a:   	3.17%
presidency:   	2.69%
--------


PROMPT: because trump 's what
he:   	23.96%
's:   	8.49%
is:   	8.12%
i:   	5.78%
the:   	5.54%
--------


PROMPT: because trump 's what makes
a:   	28.81%
him:   	10.34%
the:   	9.06%
[UNK]:   	6.29%
me:   	6.17%
--------


PROMPT: because trump 's what makes it
great:   	9.78%
a:   	8.02%
good:   	7.72%
[UNK]:   	6.36%
sad:   	6.14%
--------


PROMPT: because trump 's what makes it do
?:   	12.71%
.:   	10.19%
:   	9.01%
[UNK]:   	6.2%
the:   	5.35%
--------


PROMPT: because trump 's what makes it do ?
:   	99.28%
?:   	0.29%
[UNK]:   	0.06%
!:   	0.06%
.:   	0.05%
--------

